In [5]:
import os
import cv2
import numpy as np
from skimage import measure

# Ruta de la carpeta que contiene las imágenes
ruta_carpeta = '/home/sebastian/Documentos/analisisNETCDF/datosGraficados/2022/2022-01'

# Ruta de la carpeta donde se guardarán las imágenes binarizadas
carpeta_salida = '/home/sebastian/Documentos/analisisNETCDF/datosGraficados/2022-01_binarizado'

# Crear la carpeta de salida si no existe
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)

# Lista de archivos en la carpeta
archivos = os.listdir(ruta_carpeta)

# Variable para rastrear las imágenes procesadas
imagenes_procesadas = 0

# Iterar sobre los archivos en la carpeta
for archivo in archivos:
    # Ruta completa del archivo
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    
    # Verificar si el archivo es una imagen (puede ser útil si hay otros tipos de archivos en la carpeta)
    if not archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        continue
    
    # Lee la imagen en escala de grises
    imagen = cv2.imread(ruta_archivo, 0)
    
    if imagen is None:
        print(f"No se pudo leer la imagen: {archivo}")
        continue
    
    # Aplica umbralización para binarizar la imagen
    _, binarizada = cv2.threshold(imagen, 128, 255, cv2.THRESH_BINARY)
    
    # Invierte los colores de la imagen binarizada
    binarizada_invertida = cv2.bitwise_not(binarizada)
    
    # Encuentra los componentes conectados en la imagen binarizada invertida
    componentes = measure.label(binarizada_invertida, connectivity=2)
    
    # Encuentra el centroide del componente más grande (número en el centro)
    propiedades = measure.regionprops(componentes)
    centroide = propiedades[0].centroid
    
    # Crea una imagen en blanco y negro para mostrar los bits
    imagen_bits = np.zeros((binarizada.shape[0], binarizada.shape[1]), dtype=np.uint8)
    
    # Asigna los valores binarizados a la imagen de bits
    imagen_bits[binarizada_invertida == 255] = 1
    
    binarizada_invertida = np.bool_(binarizada_invertida)

    # Dibuja un círculo en el centroide del número
    imagen_visualizacion = cv2.cvtColor(imagen_bits * 255, cv2.COLOR_GRAY2BGR)
    cv2.circle(imagen_visualizacion, (int(centroide[1]), int(centroide[0])), 5, (0, 255, 0), -1)
    
    # Ruta de salida para la imagen binarizada
    ruta_salida = os.path.join(carpeta_salida, archivo)
    
    # Guarda la imagen binarizada en la carpeta de salida
    cv2.imwrite(ruta_salida, imagen_visualizacion)
    
    # Incrementar el contador de imágenes procesadas
    imagenes_procesadas += 1

# Verificar si faltaron imágenes por procesar
if imagenes_procesadas < len(archivos):
    print(f"Faltan {len(archivos) - imagenes_procesadas} imágenes por procesar.")

print("Proceso completo.")


Faltan 1 imágenes por procesar.
Proceso completo.
